# LangChain

In [16]:
import os
from getpass import getpass
from langchain.agents import load_tools, AgentType, initialize_agent, Tool
from langchain.llms import OpenAI
from langchain.tools import BraveSearch
from langchain.utilities import WikipediaAPIWrapper, PythonREPL

In [1]:
os.environ["OPENAI_API_KEY"] = getpass("Insert your openai api key:")

Insert your openai api key: ········


In [17]:
os.environ["BRAVE_API_KEY"] = getpass("Insert your brave api key:")

Insert your brave api key: ········


# Agentes: chamadas dinâmicas de cadeias com base na entrada do usuário

Até agora, as cadeias que examinamos seguem uma ordem predeterminada.

Os agentes não fazem isso: eles usam um LLM para determinar quais ações devem ser executadas e em que ordem. Uma ação pode usar uma ferramenta e observar sua saída ou retornar ao usuário.

Quando usados corretamente, os agentes podem ser extremamente poderosos. Neste tutorial, mostramos como usar facilmente os agentes por meio da API mais simples e de nível mais alto.

Para carregar agentes, você deve entender os seguintes conceitos:

* Ferramenta: Uma função que executa uma tarefa específica. Isso pode ser coisas como: Pesquisa do Google, pesquisa de banco de dados, Python REPL, outras cadeias. A interface de uma ferramenta é atualmente uma função que deve ter uma string como entrada, com uma string como saída.

* LLM: O modelo de linguagem que alimenta o agente.

* Agente: O agente a ser usado. Deve ser uma string que faz referência a uma classe de agente de suporte. Como este bloco de anotações se concentra na API de nível mais simples e mais alto, ele cobre apenas o uso dos agentes padrão suportados. Se você deseja implementar um agente personalizado, consulte a documentação para agentes personalizados (em breve).

In [31]:
# !pip install wikipedia

In [57]:
brave_tool = BraveSearch.from_api_key(api_key=os.environ["BRAVE_API_KEY"], search_kwargs={"count": 10})

python_repl = PythonREPL()
python_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)

In [58]:
llm = OpenAI(temperature=0.0, max_tokens=1024)
tools = [brave_tool, python_tool] + load_tools(["wikipedia", "llm-math"], llm=llm)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [39]:
agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")



> Entering new AgentExecutor chain...
 I need to find out who Leo DiCaprio's girlfriend is and then calculate her age raised to the 0.43 power.
Action: brave-search
Action Input: "Leo DiCaprio girlfriend"
Observation: [{"title": "Leo's Ladies! See Leonardo DiCaprio's Complete Dating History", "link": "https://www.usmagazine.com/celebrity-news/pictures/leonardo-dicaprios-ladies-2011911/", "snippet": "He went on to date Gisele B\u00fcndchen, Bar Refaeli, Blake Lively, Toni Garrn and Nina Agdal, among others, before finally settling down with current girlfriend <strong>Camila Morrone</strong>, who is 23 years his junior. \u201cThere\u2019s so many relationships in Hollywood \u2014 and in the history of the world \u2014 where people have large age gaps,\u201d Morrone told the Los Angeles Times in 2019 when asked whether critics of her and DiCaprio\u2019s relationship get under her skin."}, {"title": "Leonardo DiCaprio and Gigi Hadid's Relationship Timeline", "link": "https://people.com/m

"Camila Morrone is Leo DiCaprio's girlfriend and her current age raised to the 0.43 power is 3.8507291225496925."

In [55]:
agent.run("""
Gere um arquivo txt com a frase 'Olá mundo!'
""")



> Entering new AgentExecutor chain...
 I need to create a text file with the phrase 'Hello world!'
Action: python_repl
Action Input: open('hello_world.txt', 'w').write('Hello world!')
Observation: 
Thought: I now know the final answer
Final Answer: A text file named 'hello_world.txt' has been created with the phrase 'Hello world!'

> Finished chain.


"A text file named 'hello_world.txt' has been created with the phrase 'Hello world!'"

In [62]:
agent.run("Gere um arquivo txt chamado paraiba.txt com o pib e a capital da Paraíba e algumas curiosidades. Use python para gerar o arquivo")



> Entering new AgentExecutor chain...
 I need to find the PIB and capital of Paraíba, as well as some interesting facts
Action: brave-search
Action Input: "PIB Paraíba"
Observation: [{"title": "Quatro cidades concentram mais de 50% do PIB da Para\u00edba, aponta ...", "link": "https://g1.globo.com/pb/paraiba/noticia/2018/12/14/quatro-cidades-concentram-mais-de-50-do-pib-da-paraiba-aponta-ibge.ghtml", "snippet": "As cidades paraibanas com maiores PIBs, al\u00e9m de Jo\u00e3o Pessoa com R$ 18,7 bilh\u00f5es, s\u00e3o Campina Grande com R$ 8,3 bilh\u00f5es, Cabedelo com R$ 2,4 bilh\u00f5es e Santa Rita com R$ 2,1 bilh\u00f5es. As quatro juntas somam um PIB de <strong>R$ 31 bilh\u00f5es</strong>, aproximadamente 52% do total do PIB na Para\u00edba. Em 2016, 66 munic\u00edpios concentraram quase metade do PIB do pa\u00eds, diz IBGE"}, {"title": "PIB da Para\u00edba cresce quase 13% em sete anos, diz IBGE | Para\u00edba ...", "link": "https://g1.globo.com/pb/paraiba/noticia/2019/11/14/pib-

'O arquivo txt deve conter o PIB de R$ 155 143 milhões, a capital João Pessoa, e curiosidades sobre os escritores e jogadores de futebol Augusto dos Anjos, José Américo de Almeida, José Lins do Rego, Ariano Suassuna, Pedro Américo, Jaílton Paraíba e Marcelinho Paraíba.'